In [3]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets
import numpy as np

In [9]:
data = datasets.load_iris()
x_data = data.data
y_data = data.target

np.random.seed(116)
np.random.shuffle(x_data)
np.random.shuffle(y_data)

x_train = x_data[:-30]
y_train = y_data[:-30]
x_test = x_data[-30:]
y_test = y_data[-30:]

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

w1 = tf.Variable(tf.random.truncated_normal([4,3], stddev=0.1, seed=1), dtype=tf.float32)
b1 = tf.Variable(tf.random.truncated_normal([3], stddev=0.1, seed=1), dtype=tf.float32)

In [6]:
lr = 0.1
train_loss_results = []
test_acc = []
epoch = 500
loss_all = 0

In [11]:
for epoch in range(epoch):
    for step, (x_train, y_train) in enumerate(train_ds):
        with tf.GradientTape() as tape:
            y = tf.matmul(x_train, w1) + b1
            y = tf.nn.softmax(y)
            y_ = tf.one_hot(y_train, depth=3)
            loss = tf.reduce_mean(tf.math.square(y_ - y))
            loss_all += loss.numpy()

            grads = tape.gradient(loss, [w1, b1])

            w1.assign_sub(lr * grads[0])
            b1.assign_sub(lr * grads[1])
            pass
    
    print(f'Epoch {epoch}, loss{loss_all / 4.0}')
    train_loss_results.append(loss_all/4.0)

    # 测试部分
    total_correct, total_number = 0, 0
    for x_test, y_test in test_ds:
        y = tf.matmul(x_test, w1) + b1
        y = tf.math.softmax(y)
        pred = tf.math.argmax(y, axis = 1)
        pred = tf.cast(pred, dtype=y_test.dtype)
        correct = tf.cast(tf.math.equal(pred, y_test), dtype=tf.int32)
        correct = tf.math.reduce_sum(correct)
        total_correct += int(currect)
        total_number += x_test.shape[0]
        pass
    
    acc = total_correct / total_number
    test_acc.append(acc)
    print(f'Test_acc: {acc}')
    print('-----------------')

